In [1]:
import platform 
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("MyApp")
        .config("hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.warehouse.dir","hdfs://localhost:9000/users/hive/warehouse")
        .config("spark.driver.memory","2g")
        .enableHiveSupport()
        .getOrCreate())
    
    # spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 02:27:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
spark.stop()

In [2]:
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [1]:
!hdfs dfs -put /usr/notebooks/Farmacias.csv /user/Daniel

In [3]:
!hdfs dfs -ls /user/Daniel/

Found 1 items
-rw-r--r--   1 Daniel root        758 2024-06-26 02:05 /user/Daniel/Farmacias.csv


In [5]:
spark.sql("show databases").show()

24/06/26 02:27:48 WARN HiveClientImpl: HiveClient got thrift exception, destroying client and retrying (0 tries remaining)
org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.metadata.Hive.getDatabase(Hive.java:1567)
	at org.apache.hadoop.hive.ql.metadata.Hive.databaseExists(Hive.java:1552)
	at org.apache.spark.sql.hive.client.Shim_v0_12.databaseExists(HiveShim.scala:609)
	at org.apache.spark.sql.hive.client.HiveClientImpl.$anonfun$databaseExists$1(HiveClientImpl.scala:406)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at org.apache.spark.sql.hive.client.HiveClientImpl.$anonfun$withHiveState$1(HiveClientImpl.scala:303)
	at org.apache.spark.sql.hive.client.HiveClientImpl.liftedTree1$1(HiveClientImpl.scala:234)
	at org.apache.spark.sql.hive.client.HiveClientImpl.retryLocked(HiveClientImpl.scala:233)
	at org.apache.spark.

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [4]:
df = spark.read.csv("/user/Daniel/Farmacias.csv",sep=";",header=True)
df.show()

+--------------------+-------------+-----------------------+---------------+
|              CIDADE|FLAG_SHOPPING|SEGUNDO_PRINCIPAL_SETOR|FATURAMENTO_TRI|
+--------------------+-------------+-----------------------+---------------+
|           São Paulo|          Sim|             Cosméticos|           34,8|
|            Campinas|          Sim|             Cosméticos|           28,6|
|           Guarulhos|          Sim|        Higiene pessoal|           27,3|
|              Osasco|          Sim|             Perfumaria|           24,5|
|São Bernardo do C...|          Sim|             Cosméticos|           24,1|
|      Ribeirão Preto|          Sim|             Cosméticos|           23,6|
|             Diadema|          Não|             Perfumaria|           21,8|
|            Sorocaba|          Sim|             Cosméticos|           19,8|
|        Praia Grande|          Não|        Higiene pessoal|           19,4|
|             Jundiaí|          Não|             Perfumaria|           18,5|

In [3]:
spark.sql('''

select 

--substring(UF_zi,1,2) as UF,
--count (distinct ano_cmpt),
--count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from bronze.rd_sih

--group by 1
--order by 1
''').show(30)

+--------------------------------------+--------+
|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+--------------------------------------+--------+
|                                299648|  299648|
+--------------------------------------+--------+



In [4]:
df = spark.read.csv('/DataSus/rd/csv',sep=";",header=True)
df.createOrReplaceTempView("test")

spark.sql('''

select 

--substring(UF_zi,1,2) as UF,
--count (distinct ano_cmpt),
--count (distinct mes_cmpt),
count (distinct N_AIH,DT_SAIDA,IDENT),
count(*)

from test

--group by 1
--order by 1
''').show(30)

+--------------------------------------+--------+
|count(DISTINCT N_AIH, DT_SAIDA, IDENT)|count(1)|
+--------------------------------------+--------+
|                                299648|  345124|
+--------------------------------------+--------+



In [20]:
df = pd.DataFrame({'name': ['Raphael', 'Donatello'],
                   'mask': ['red', 'purple'],
                   'weapon': ['sai', 'bo staff']})


In [25]:

df.to_csv('df.csv')